In [1]:
import torch
# data loading
import pandas as pd
from plyfile import PlyData
# plotting
from meshplot import plot
from ipywidgets import interact, FloatSlider

# implementation
from implementation import minmax_scaling, centerzoom_scaling, DPSR_forward
from skimage.measure import marching_cubes

### Load data and set parameters

In [2]:
SCAN = "./deep_geometric_prior_data/scans/dc.ply"

## configuration
GRID = tuple([256] * 3)
SIGMA = 1
# authors use a "normalization" + sigmoid schema as preprocessing
# anything can be used as long as the final space in which points
# reside in the interval (0,1) along each dimension
PREPROC = lambda x: torch.sigmoid(centerzoom_scaling(x, scale=0.9))
# we tried min-max scaling with noticeably worse results then the above
# PREPROC = lambda x: torch.sigmoid(minmax_scaling(x, eps=1e-5))

In [3]:
def read_ply_file(path):
    data = PlyData.read(path)

    xyz = [torch.tensor(data["vertex"][axis]) for axis in ["x", "y", "z"]]
    xyz = torch.stack(xyz, dim=-1)

    nxyz = [torch.tensor(data["vertex"][axis]) for axis in ["nx", "ny", "nz"]]
    nxyz = torch.stack(nxyz, dim=-1)
    return xyz.unsqueeze(0), nxyz.unsqueeze(0)

In [4]:
points, normals = read_ply_file(SCAN)

### Pointcloud with normals

In [5]:
plot(minmax_scaling(points[0], eps=0).numpy(), c=normals[0].numpy());

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.50…

In [6]:
# compute indicator function
eps = 1e-6  # division by zero
m = 0.5     # indicator scaling

class SelfMockup:
    pass

mockup = SelfMockup()
mockup.grid = GRID
mockup.sigma = SIGMA
mockup.eps = eps
mockup.m = m
chi = DPSR_forward(mockup, PREPROC(points), normals)

### Mesh reconstruction with indicator $\chi$ and marching cubes

In [7]:
# reconstruct mesh
v, f, n, val = marching_cubes(chi[0].numpy(), level=0)
plot(v=v,f=f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.37522…